# 🎉 Animations

---
## 🔍 Notebook objectives


This notebook contains simulation of attractor state in the ring attractor network which contains 360 excitatory RS neurons and 90 inhibitory FS interneurons for different serotonin levels to create interactive animations of system dynamics.

# 🎒 Setup

## ⬇️ Imports

In [ ]:
from utils.main_simulation import network_simulation_run
from utils.poisson_spike_train_generation import generate_spike_train
from utils.plots import *
from plotly_gif import GIF, capture

import numpy as np
import yaml #reading env consts

## 🛠️ Simulation Utils

In [ ]:
#for calculating stopping time
def calculate_instantaneous_firing_rate(firings, window_size=5000, threshold_hz = 12, burst_threshold_hz = 100):
    num_time_points, num_of_neurons = firings.shape
    half_window = window_size // 2
    firing_rates = np.zeros(num_time_points)

    for t in range(half_window, num_time_points - half_window, 500):
        window_start = t - half_window
        window_end = t + half_window
        window_spikes = firings[window_start:window_end + 1, :]
        total_spikes = np.sum(window_spikes, axis=0)
        firing_rate = total_spikes * 10000 / window_size
        if np.any(firing_rate > burst_threshold_hz):
            return t / 10000
        elif np.any(firing_rate > threshold_hz):
            continue
        else:
            return t / 10000
    return num_time_points / 10000

In [ ]:
#RS neuron parameters
with open('utils/RS.yaml', 'r', encoding="utf-8") as f:
    params_RS = yaml.load(f, Loader=yaml.FullLoader)

#FS neuron parameters
with open('utils/FS.yaml', 'r', encoding="utf-8") as f:
    params_FS = yaml.load(f, Loader=yaml.FullLoader)

#receptor kinetics parameters
with open('utils/receptor_kinetics.yaml', 'r', encoding="utf-8") as f:
    params_receptor_kinetics = yaml.load(f, Loader=yaml.FullLoader)

#synaptic weights parameters
with open('utils/synaptic_weights.yaml', 'r', encoding="utf-8") as f:
    params_synaptic_weights = yaml.load(f, Loader=yaml.FullLoader)

#external input parameters
with open('utils/external_input.yaml', 'r', encoding="utf-8") as f:
    params_external_input = yaml.load(f, Loader=yaml.FullLoader)

#stimuli input parameters
with open('utils/stimuli_input.yaml', 'r', encoding="utf-8") as f:
    params_stimuli_input = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
#simulation time; always 5 seconds, for all experiments
seconds = 5

t_min = 0
t_max = int(seconds*1000) #in ms -> 1(s) of simulation
delta_T = 0.1 #0.1 ms is integration step
sim_steps = int(seconds*1000/delta_T)

T = np.linspace(t_min, t_max, sim_steps)

# 🧪 Experiment

### 🏋️ Synaptic Weights

Definition of synaptic weights.

In [ ]:
#360 RS and 90 FS
Ne = 360
Ni = 90

weights = np.array(np.zeros((Ne + Ni, Ne + Ni))) #matrix (Ne + Ni) x (Ne + Ni) (i, j: i -> j), thus sum by column for input of j

# e -> e
for i in range(Ne):
    for j in range(Ne):
        angle_i = i / Ne * 360
        angle_j = j / Ne * 360
        weights[i, j] = params_synaptic_weights['j_ee'] * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/params_synaptic_weights['sigma']**2)

# e -> i
for i in range(Ne):
    for j in range(Ni):
        angle_i = i / Ne * 360
        angle_j = j / Ni * 360
        weights[i, Ne + j] = params_synaptic_weights['j_ei']

# i -> e
for i in range(Ni):
    for j in range(Ne):
        angle_i = i / Ni * 360
        angle_j = j / Ne * 360
        weights[Ne + i, j] = params_synaptic_weights['j_ie']
        
# i -> i
for i in range(Ni):
    for j in range(Ni):
        angle_i = i / Ni * 360
        angle_j = j / Ni * 360
        weights[Ne + i, Ne + j] = params_synaptic_weights['j_ii']

### 🚝 External Cortical Input & 🎯 Stimulus Representation

Notice that this cell has received additional developments in comparison with `spontaneous_state.ipynb` notebook. Now, we define stimuli parameters and experiment timings.

In [ ]:
v = np.array(np.zeros((len(T), Ne + Ni)))
v[:, :Ne] = params_external_input['v_e']
v[:, Ne:] = params_external_input['v_i']

eta = params_stimuli_input["eta"]
A = params_stimuli_input["A"]

t_stim_start = 1
t_stim_end = 1.25
delay_time = 3.75

### 🥳 Serotonin Weights

Still, no serotonin weights for now, follow the final notebook `attractor_state_serotonin.ipynb` for more.

In [ ]:
serotonin_weights = np.ones(Ne + Ni)

### 🔩 Settings

In [ ]:
start_state = np.column_stack((np.append(params_RS["v"]*np.ones(Ne), params_FS["v"]*np.ones(Ni)), np.append(params_RS["u"]*np.ones(Ne), params_FS["u"]*np.ones(Ni)))) #matrix (Ne + Ni) x 2 - (v, u) for each neuron

params_network = {"a": np.append(params_RS["a"]*np.ones(Ne), params_FS["a"]*np.ones(Ni)), 
          "b": np.append(params_RS["b"]*np.ones(Ne), params_FS["b"]*np.ones(Ni)), 
          "c": np.append(params_RS["c"]*np.ones(Ne), params_FS["c"]*np.ones(Ni)), 
          "d": np.append(params_RS["d"]*np.ones(Ne), params_FS["d"]*np.ones(Ni)), 
          "C": np.append(params_RS["C"]*np.ones(Ne), params_FS["C"]*np.ones(Ni)), 
          "k": np.append(params_RS["k"]*np.ones(Ne), params_FS["k"]*np.ones(Ni)),
          "v_peak": np.append(params_RS["v_peak"]*np.ones(Ne), params_FS["v_peak"]*np.ones(Ni)), 
          "v_r": np.append(params_RS["v_r"]*np.ones(Ne), params_FS["v_r"]*np.ones(Ni)), 
          "v_t": np.append(params_RS["v_t"]*np.ones(Ne), params_FS["v_t"]*np.ones(Ni)),
          "tau_ampa": params_receptor_kinetics["tau_ampa"]*np.ones((Ne + Ni)),
          "tau_nmda": params_receptor_kinetics["tau_nmda"]*np.ones((Ne + Ni)),
          "tau_gabaa": params_receptor_kinetics["tau_gabaa"]*np.ones((Ne + Ni)),
          "tau_gabab": params_receptor_kinetics["tau_gabab"]*np.ones((Ne + Ni)),
          "g_ampa": params_receptor_kinetics["g_ampa"]*np.ones((Ne + Ni)),
          "g_nmda": params_receptor_kinetics["g_nmda"]*np.ones((Ne + Ni)),
          "g_gabaa": params_receptor_kinetics["g_gabaa"]*np.ones((Ne + Ni)),
          "g_gabab": params_receptor_kinetics["g_gabab"]*np.ones((Ne + Ni)),
          "g_e_external": params_external_input["g_e_external"]*np.ones((Ne + Ni)),
          "g_i_external": params_external_input["g_i_external"]*np.ones((Ne + Ni))
}

### 🟰 Normal Serotonin Level

In [ ]:
np.random.seed(42) #for reproducibility

#random selection of stimuli orientation
stimuli_orientation = 100
serotonin_weights = np.ones(Ne + Ni)

#present stimuli to the network
for i in range(Ne):
    angle_i = i / Ne * 360
    v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

#simulation itself
states, firings, synaptic_input, synaptic_inputs, background_input, conductances, background_conductance = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)
termination_time = calculate_instantaneous_firing_rate(firings[int(t_stim_end * 1000 / delta_T):, :360], window_size = 5000)

exc_firing_indices = np.argwhere(firings[:, :360] == 1)
inh_firing_indices = np.argwhere(firings[:, 360:] == 1)

In [ ]:
# @markdown Generate GIF

gif = GIF(verbose=True, gif_path = "images/")
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Excitatory neurons spikes", "Inhibitory neurons spikes"))

fig.update_layout(
    xaxis_title='Time (in s)',
    xaxis_range=[0, 5],
    yaxis_range=[0, 360],
    xaxis2_title='Time (in s)',
    xaxis2_range=[0, 5],
    yaxis2_range=[0, 90],
    yaxis_title='Neuron',
    title="Normal Serotonin",
    yaxis2_title='Neuron',
    showlegend = False,
    height = 600,
    width = 1200
    )


@capture(gif)  # tells gif to save each figure that is generated from this function
def plot_(fig, exc_firing_indices, inh_firing_indices, stimulation_start, stimulation_end):
    trace_1 = go.Scatter(x=exc_firing_indices[:, 0] * delta_T / 1000, y=exc_firing_indices[:, 1], mode='markers', marker=dict(color='blue', size=4))
    trace_2 = go.Scatter(x=inh_firing_indices[:, 0] * delta_T / 1000, y=inh_firing_indices[:, 1], mode='markers', marker=dict(color='red', size=4))
    fig.add_trace(trace_1, row=1, col=1)
    fig.add_trace(trace_2, row=2, col=1)
    fig.add_shape(
        type="line",
        x0=stimulation_start,
        x1=stimulation_end,
        y0=-0.05,
        y1=-0.05,
        line=dict(
        color="blue",
        width=5,
        ),
        yref="paper",
    )  
    return fig

frames = 50
sim_start = 0
sim_end = 0
for i in range(frames):
    # Changes the range of the data each step to make it look like a time series
    if i >= 10 and i < 12.5:
        sim_start = 1
        sim_end = i / 10
    elif i >= 12.5:
        sim_start = 1
        sim_end = 1.25
        
    fig = plot_(fig, exc_firing_indices[np.abs(exc_firing_indices[:, 0] - 1000*i - 500) <= 500, :], inh_firing_indices[np.abs(inh_firing_indices[:, 0] - 1000*i - 500) <= 500, :], sim_start, sim_end)

# Create gif
gif.create_gif(length = 5000, gif_path = "images/normal_serotonin.gif")

### 📈 Increased Serotonin Levels

In [ ]:
np.random.seed(42) #for reproducibility

#random selection of stimuli orientation
stimuli_orientation = 100
serotonin_weights = np.ones(Ne + Ni)
serotonin_weights[:Ne] = 0.97*serotonin_weights[:Ne] 

#present stimuli to the network
for i in range(Ne):
    angle_i = i / Ne * 360
    v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

#simulation itself
states, firings, synaptic_input, synaptic_inputs, background_input, conductances, background_conductance = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)

termination_time = calculate_instantaneous_firing_rate(firings[int(t_stim_end * 1000 / delta_T):, :360], window_size = 5000)
print(f"Termination time for attractor state is: {termination_time:.02f}")

In [ ]:
# @markdown Generate GIF

exc_firing_indices = np.argwhere(firings[:, :360] == 1)
inh_firing_indices = np.argwhere(firings[:, 360:] == 1)

gif = GIF(verbose=True, gif_path = "images/")
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Excitatory neurons spikes", "Inhibitory neurons spikes"))

fig.update_layout(
    xaxis_title='Time (in s)',
    xaxis_range=[0, 5],
    yaxis_range=[0, 360],
    xaxis2_title='Time (in s)',
    xaxis2_range=[0, 5],
    yaxis2_range=[0, 90],
    yaxis_title='Neuron',
    title="Increased Serotonin",
    yaxis2_title='Neuron',
    showlegend = False,
    height = 600,
    width = 1200
    )


@capture(gif)  # tells gif to save each figure that is generated from this function
def plot_(fig, exc_firing_indices, inh_firing_indices, stimulation_start, stimulation_end):
    trace_1 = go.Scatter(x=exc_firing_indices[:, 0] * delta_T / 1000, y=exc_firing_indices[:, 1], mode='markers', marker=dict(color='blue', size=4))
    trace_2 = go.Scatter(x=inh_firing_indices[:, 0] * delta_T / 1000, y=inh_firing_indices[:, 1], mode='markers', marker=dict(color='red', size=4))
    fig.add_trace(trace_1, row=1, col=1)
    fig.add_trace(trace_2, row=2, col=1)
    fig.add_shape(
        type="line",
        x0=stimulation_start,
        x1=stimulation_end,
        y0=-0.05,
        y1=-0.05,
        line=dict(
        color="blue",
        width=5,
        ),
        yref="paper",
    )  
    return fig

frames = 50
sim_start = 0
sim_end = 0
for i in range(frames):
    # Changes the range of the data each step to make it look like a time series
    if i >= 10 and i < 12.5:
        sim_start = 1
        sim_end = i / 10
    elif i >= 12.5:
        sim_start = 1
        sim_end = 1.25
        
    fig = plot_(fig, exc_firing_indices[np.abs(exc_firing_indices[:, 0] - 1000*i - 500) <= 500, :], inh_firing_indices[np.abs(inh_firing_indices[:, 0] - 1000*i - 500) <= 500, :], sim_start, sim_end)

# Create gif
gif.create_gif(length = 5000, gif_path = "images/increased_serotonin.gif")

### 📉 Decreased Serotonin Levels

In [ ]:
np.random.seed(42) #for reproducibility

#random selection of stimuli orientation
stimuli_orientation = 100
serotonin_weights = 1.02*np.ones(Ne + Ni)

#present stimuli to the network
for i in range(Ne):
    angle_i = i / Ne * 360
    v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

#simulation itself
states, firings, synaptic_input, synaptic_inputs, background_input, conductances, background_conductance = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)
termination_time = calculate_instantaneous_firing_rate(firings[int(t_stim_end * 1000 / delta_T):, :360], window_size = 5000)

exc_firing_indices = np.argwhere(firings[:, :360] == 1)
inh_firing_indices = np.argwhere(firings[:, 360:] == 1)

In [ ]:
# @markdown Generate GIF

gif = GIF(verbose=True, gif_path = "images/")
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Excitatory neurons spikes", "Inhibitory neurons spikes"))

fig.update_layout(
    xaxis_title='Time (in s)',
    xaxis_range=[0, 5],
    yaxis_range=[0, 360],
    xaxis2_title='Time (in s)',
    xaxis2_range=[0, 5],
    yaxis2_range=[0, 90],
    yaxis_title='Neuron',
    title="Decreased Serotonin",
    yaxis2_title='Neuron',
    showlegend = False,
    height = 600,
    width = 1200
    )


@capture(gif)  # tells gif to save each figure that is generated from this function
def plot_(fig, exc_firing_indices, inh_firing_indices, stimulation_start, stimulation_end):
    trace_1 = go.Scatter(x=exc_firing_indices[:, 0] * delta_T / 1000, y=exc_firing_indices[:, 1], mode='markers', marker=dict(color='blue', size=4))
    trace_2 = go.Scatter(x=inh_firing_indices[:, 0] * delta_T / 1000, y=inh_firing_indices[:, 1], mode='markers', marker=dict(color='red', size=4))
    fig.add_trace(trace_1, row=1, col=1)
    fig.add_trace(trace_2, row=2, col=1)
    fig.add_shape(
        type="line",
        x0=stimulation_start,
        x1=stimulation_end,
        y0=-0.05,
        y1=-0.05,
        line=dict(
        color="blue",
        width=5,
        ),
        yref="paper",
    )  
    return fig

frames = 50
sim_start = 0
sim_end = 0
for i in range(frames):
    # Changes the range of the data each step to make it look like a time series
    if i >= 10 and i < 12.5:
        sim_start = 1
        sim_end = i / 10
    elif i >= 12.5:
        sim_start = 1
        sim_end = 1.25
        
    fig = plot_(fig, exc_firing_indices[np.abs(exc_firing_indices[:, 0] - 1000*i - 500) <= 500, :], inh_firing_indices[np.abs(inh_firing_indices[:, 0] - 1000*i - 500) <= 500, :], sim_start, sim_end)

# Create gif
gif.create_gif(length = 5000, gif_path = "images/decreased_serotonin.gif")

### 🔀 Slightly Decreased Serotonin Levels

In [ ]:
np.random.seed(42) #for reproducibility

#random selection of stimuli orientation
stimuli_orientation = 100
serotonin_weights = 1.01*np.ones(Ne + Ni)

#present stimuli to the network
for i in range(Ne):
    angle_i = i / Ne * 360
    v[int(t_stim_start*len(T)/seconds):int(t_stim_end*len(T)/seconds), i] = params_external_input['v_e'] * (1 + A * np.exp(eta * (np.cos(np.deg2rad(angle_i) - np.deg2rad(stimuli_orientation)) - 1)))
spike_trains = generate_spike_train(Ne, Ni, v, T, delta_T)

#simulation itself
states, firings, synaptic_input, synaptic_inputs, background_input, conductances, background_conductance = network_simulation_run(Ne, Ni, T, delta_T, start_state, weights, spike_trains, serotonin_weights, params_network)
termination_time = calculate_instantaneous_firing_rate(firings[int(t_stim_end * 1000 / delta_T):, :360], window_size = 5000)

exc_firing_indices = np.argwhere(firings[:, :360] == 1)
inh_firing_indices = np.argwhere(firings[:, 360:] == 1)

In [ ]:
# @markdown Generate GIF

gif = GIF(verbose=True, gif_path = "images/")
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Excitatory neurons spikes", "Inhibitory neurons spikes"))

fig.update_layout(
    xaxis_title='Time (in s)',
    xaxis_range=[0, 5],
    yaxis_range=[0, 360],
    xaxis2_title='Time (in s)',
    xaxis2_range=[0, 5],
    yaxis2_range=[0, 90],
    yaxis_title='Neuron',
    title="Slightly Decreased Serotonin",
    yaxis2_title='Neuron',
    showlegend = False,
    height = 600,
    width = 1200
    )


@capture(gif)  # tells gif to save each figure that is generated from this function
def plot_(fig, exc_firing_indices, inh_firing_indices, stimulation_start, stimulation_end):
    trace_1 = go.Scatter(x=exc_firing_indices[:, 0] * delta_T / 1000, y=exc_firing_indices[:, 1], mode='markers', marker=dict(color='blue', size=4))
    trace_2 = go.Scatter(x=inh_firing_indices[:, 0] * delta_T / 1000, y=inh_firing_indices[:, 1], mode='markers', marker=dict(color='red', size=4))
    fig.add_trace(trace_1, row=1, col=1)
    fig.add_trace(trace_2, row=2, col=1)
    fig.add_shape(
        type="line",
        x0=stimulation_start,
        x1=stimulation_end,
        y0=-0.05,
        y1=-0.05,
        line=dict(
        color="blue",
        width=5,
        ),
        yref="paper",
    )  
    return fig

frames = 50
sim_start = 0
sim_end = 0
for i in range(frames):
    # Changes the range of the data each step to make it look like a time series
    if i >= 10 and i < 12.5:
        sim_start = 1
        sim_end = i / 10
    elif i >= 12.5:
        sim_start = 1
        sim_end = 1.25
        
    fig = plot_(fig, exc_firing_indices[np.abs(exc_firing_indices[:, 0] - 1000*i - 500) <= 500, :], inh_firing_indices[np.abs(inh_firing_indices[:, 0] - 1000*i - 500) <= 500, :], sim_start, sim_end)

# Create gif
gif.create_gif(length = 5000, gif_path = "images/slightly_decreased_serotonin.gif")